<a href="https://colab.research.google.com/github/flatfeetpete/nerfstudio/blob/main/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [1]:
#@markdown <h1>Install Conda (requires runtime restart)</h1>

!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
#@markdown <h1>Install Nerfstudio and Dependencies (~15 min)</h1>

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=12RL_NVgE9WGvr_fEsXEiuaJ1QESvQCPl&confirm=t" 
!pip install tinycudann-1.7-cp38-cp38-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
# !pip install nerfstudio
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git
!conda remove --force qt-main

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp38-cp38-linux_x86_64.whl (1837.7 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp38-cp38-linux_x86_64.whl (23.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1
    Uninstalling torchvision-0.14.1:
      Successfully uninstalled torchvision-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [3]:
 scene = "custom_data"

In [ ]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDE9C dozer' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content
--2023-02-26 20:21:20--  https://data.nerf.studio/nerfstudio/dozer.zip
Resolving data.nerf.studio (data.nerf.studio)... 34.102.68.79
Connecting to data.nerf.studio (data.nerf.studio)|34.102.68.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1347483375 (1.3G) [application/zip]
Saving to: ‘data/nerfstudio/dozer.zip’

data/nerfstudio/doz 100%[===================>]   1.25G  1.74MB/s    in 4m 13s  

2023-02-26 20:25:34 (5.08 MB/s) - ‘data/nerfstudio/dozer.zip’ saved [1347483375/1347483375]

Data Processing Succeeded!


In [19]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 3.875s
https://viewer.nerf.studio/?websocket_url=wss://violet-rules-arrive-34-147-60-67.loca.lt
You may need to click Refresh Page after you start training!


In [ ]:
#@markdown <h1>Start Training</h1>

import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# This restarts training, but only works if you finished training on your first run. You can use the UI to continue traiing if it didn't finish.
%cd /content
#!echo $(ls -rt outputs/data-nerfstudio-$scene/nerfacto | head -1)
#!ns-train nerfacto --viewer.websocket-port 7007 --load-dir=outputs/data-nerfstudio-$scene/nerfacto/$(ls -rt outputs/data-nerfstudio-$scene/nerfacto | head -1)/nerfstudio_models --viewer.start-train=False nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4

#!ns-train nerfacto --viewer.websocket-port 7007 --load-dir=/content/outputs/unnamed/nerfacto/2023-03-01_034009/nerfstudio_models --viewer.start-train=False nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
!ns-train nerfacto --viewer.websocket-port 7007 --load-dir=/content/outputs/unnamed/nerfacto/2023-03-04_232541/nerfstudio_models --viewer.start-train=False nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4




/content
──────────────────────────────────────────────────────── Config ────────────────────────────────────────────────────────
TrainerConfig(
    _target=<class 'nerfstudio.engine.trainer.Trainer'>,
    output_dir=PosixPath('outputs'),
    method_name='nerfacto',
    experiment_name=None,
    timestamp='2023-03-05_004109',
    machine=MachineConfig(seed=42, num_gpus=1, num_machines=1, machine_rank=0, dist_url='auto'),
    logging=LoggingConfig(
        relative_log_dir=PosixPath('.'),
        steps_per_log=10,
        max_buffer_size=20,
        local_writer=LocalWriterConfig(
            _target=<class 'nerfstudio.utils.writer.LocalWriter'>,
            enable=True,
            stats_to_track=(
                <EventName.ITER_TRAIN_TIME: 'Train Iter (time)'>,
                <EventName.TRAIN_RAYS_PER_SEC: 'Train Rays / Sec'>,
                <EventName.CURR_TEST_PSNR: 'Test PSNR'>,
                <EventName.VIS_RAYS_PER_SEC: 'Vis Rays / Sec'>,
                <EventName.TEST_RAYS_

In [30]:
%cd /content/nerf-work/
!git pull
%cd /content/nerfstudio/
!git pull

/content/nerf-work
Already up to date.
/content/nerfstudio
Already up to date.


In [32]:
import datetime
import pytz
%cd /content/nerf-work/
for i in range(60):
  !git add --all
  !git commit -a -m 'dummy commit message (likely test files)'
  !git push
  print(datetime.datetime.now(pytz.timezone('America/Los_Angeles')))
  time.sleep(300)


/content/nerf-work
[main 9cadc3d] dummy commit message (likely test files)
 8 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00038.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00039.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00040.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00041.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00042.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00043.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00044.png
 create mode 100644 20230304-60fps-dragon-ed0.007.images/00045.png
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 19.76 MiB | 6.02 MiB/s, done.
Total 11 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:flatf

KeyboardInterrupt: ignored

In [15]:
# !ns-render --load-config outputs/data-nerfstudio-dozer/nerfacto/2023-02-28_010246/config.yml --traj filename --camera-path-filename data/nerfstudio/dozer/camera_paths/2023-02-28_010246.json --output-path renders/dozer/2023-02-28_010246.mp4
#%cd /content
#!cp data/nerfstudio/dozer/camera_paths/2023-02-28_010246.json nerf-work/longer_path.json
# --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-01_065124.json --output-path renders/custom_data/2023-03-01_065124.mp4

# export PYTHONPATH=/content/nerfstudio/:/env/python; python3 /content/nerfstudio/scripts/render.py --load-config outputs/unnamed/nerfacto/2023-03-01_065124/config.yml --traj filename --camera-path-filename nerf-work/2023-03-01_065124.json --output-path /content/nerf-work/20230228-long-dragon-0.03.mp4

#export PYTHONPATH=/content/nerfstudio/:/env/python; python3 /content/nerfstudio/scripts/render.py --load-config outputs/unnamed/nerfacto/2023-03-01_034009/config.yml --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-01_034009.json --output-path /content/nerf-works/20230228-dragon-lowfps.mp4
ns-render --load-config outputs/unnamed/nerfacto/2023-03-05_004109/config.yml --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-05_004109.json --output-path /content/nerf-work/20230304-short-back-to-teacher.mp4

#export PYTHONPATH=/content/nerfstudio/:/env/python; python3 /content/nerfstudio/scripts/train.py nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4

ns-render --load-config outputs/unnamed/nerfacto/2023-03-05_004109/config.yml --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-05_004109.json --output-path /content/nerf-work/20230304-60fps-dragon-ed.mp4

# smapller just to see

ns-render


export PYTHONPATH=/content/nerfstudio/:/env/python; python3 /content/nerfstudio/scripts/render.py  \
 --load-config outputs/unnamed/nerfacto/2023-03-05_004109/config.yml \
 --traj filename \
 --camera-path-filename /content/nerf-work/2023-03-04-in_classroom_60fps.json \
 --output-path /content/nerf-work/20230304-60fps-dragon-ed.images.png \
 --output-format images \
 --seconds 0.05


SyntaxError: ignored

In [26]:
%cd /content
#WORKED !ffmpeg -i /content/nerf-work/20230227_dozer_2d_sbs_orbit_ed0.05.mp4 -vcodec libx264 -x264opts "frame-packing=3" /content/nerf-work/20230227_dozer_2d_sbs_orbit_ed0.05.sbsembed.mp4
!ffmpeg -i /content/nerf-work/20230304-60fps-dragon-ed.images/%05d.png -vcodec libx264 -x264opts "frame-packing=3" /content/nerf-work/2023-03-04-class-truncation-ed0.03.sbsembed.mp4

#!ffmpeg -stream_loop 3 -i /content/nerf-work/20230228-long-dragon-0.03.mp4 -vcodec libx264 -x264opts "frame-packing=3" /content/nerf-work/20230228-long-dragon-0.03.tagged.looped3.mp4

# FAILED  !ffmpeg -i /content/nerf-work/20230227_dozer_2d_sbs_orbit_ed0.05.mp4 -c copy -metadata:s:v:0 stereo_mode=1 /content/nerf-work/20230227_sbs001.mkv 

/content
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libv

In [ ]:
%cd /content/nerfstudio
!git commit -a -m "fix ipd hacks"
!git push

/content/nerfstudio
[main 75decf4] fix ipd hacks
 1 file changed, 2 insertions(+), 1 deletion(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 449 bytes | 449.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To github.com:flatfeetpete/nerfstudio.git
   31a827d..75decf4  main -> main


In [18]:
%cd /content
!ns-render --load-config outputs/unnamed/nerfacto/2023-03-04_232541/config.yml --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-04_232541.json --output-path nerf-work/20230304-short_dragon_aisle.mp4

/content
[00:08:12] Skipping 0 files in dataset split train.                                         ]8;id=145696;file:///usr/local/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=970831;file:///usr/local/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#165\165]8;;\
            Dataset is overriding orientation method to none                                ]8;id=794480;file:///usr/local/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=850639;file:///usr/local/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#204\204]8;;\
           Skipping 0 files in dataset split test.                                          ]8;id=792447;file:///usr/local/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=11369;file:/